## Expert Knowledge Worker

### A question answering agent that is an expert knowledge worker
### To be used by employees of Insurellm, an Insurance Tech company
### The agent needs to be accurate and the solution should be low cost.

This project will use RAG (Retrieval Augmented Generation) to ensure our question/answering assistant has high accuracy.

In [11]:
# imports

import os
import glob
from dotenv import load_dotenv
import gradio as gr

In [12]:
# imports for langchain

from langchain.document_loaders import DirectoryLoader, TextLoader
from langchain.text_splitter import CharacterTextSplitter

In [13]:
# price is a factor for our company, so we're going to use a low cost model

MODEL = "gpt-4o-mini"
db_name = "vector_db"
prefix_path = '/app/jupyter_notebook_ai_clone/llm_udemy/llm_engineering/week5/'

In [14]:
# Load environment variables in a file called .env

load_dotenv()
os.environ['OPENAI_API_KEY'] = os.getenv('OPENAI_API_KEY', 'your-key-if-not-using-env')

In [15]:
# Read in documents using LangChain's loaders
# Take everything in all the sub-folders of our knowledgebase

folders = glob.glob(f"{prefix_path}knowledge-base/*")

# With thanks to CG and Jon R, students on the course, for this fix needed for some users 
text_loader_kwargs = {'encoding': 'utf-8'}
# If that doesn't work, some Windows users might need to uncomment the next line instead
# text_loader_kwargs={'autodetect_encoding': True}

documents = []
for folder in folders:
    doc_type = os.path.basename(folder)
    loader = DirectoryLoader(folder, glob="**/*.md", loader_cls=TextLoader, loader_kwargs=text_loader_kwargs)
    folder_docs = loader.load()
    for doc in folder_docs:
        doc.metadata["doc_type"] = doc_type
        documents.append(doc)

In [16]:
len(documents)

31

In [17]:
documents[24]

Document(metadata={'source': '/app/jupyter_notebook_ai_clone/llm_udemy/llm_engineering/week5/knowledge-base/contracts/Contract with Roadway Insurance Inc. for Carllm.md', 'doc_type': 'contracts'}, page_content="# Contract with Roadway Insurance Inc. for Carllm\n\n---\n\n## Terms\n\n1. **Agreement Effective Date**: This contract is effective as of January 1, 2025.\n2. **Duration**: This agreement will remain in effect for a term of 12 months, concluding on December 31, 2025.\n3. **Subscription Type**: Roadway Insurance Inc. agrees to subscribe to the **Professional Tier** of Carllm, at a cost of $2,500/month, totaling $30,000 for the duration of this contract.\n4. **Payment Terms**: Payments are due on the first of each month. Late payments will incur a penalty of 1.5% per month.\n5. **Termination Clause**: Either party may terminate this agreement with 30 days' written notice prior to the end of the term. If terminated early, fees will be calculated on a pro-rata basis.\n\n---\n\n## Re

In [18]:
text_splitter = CharacterTextSplitter(chunk_size=1000, chunk_overlap=200)
chunks = text_splitter.split_documents(documents)

Created a chunk of size 1088, which is longer than the specified 1000


In [19]:
len(chunks)

123

In [20]:
chunks[6]

Document(metadata={'source': '/app/jupyter_notebook_ai_clone/llm_udemy/llm_engineering/week5/knowledge-base/employees/Emily Carter.md', 'doc_type': 'employees'}, page_content='## Compensation History\n| Year | Base Salary | Bonus         | Total Compensation |\n|------|-------------|---------------|--------------------|\n| 2023 | $70,000     | $10,000       | $80,000            |\n| 2022 | $65,000     | $8,000        | $73,000            |\n| 2021 | $60,000     | $5,000        | $65,000            |\n\n## Other HR Notes\n- **Professional Development:** Emily is currently enrolled in a leadership training program to enhance her management skills and aims to move into a senior account role within the next 2 years.  \n- **Volunteer Work:** Actively participates in community outreach programs, representing Insurellm in charity events to promote corporate social responsibility.  \n- **Interests:** In her spare time, Emily enjoys hiking, photography, and volunteering at local animal shelters

In [21]:
doc_types = set(chunk.metadata['doc_type'] for chunk in chunks)
print(f"Document types found: {', '.join(doc_types)}")

Document types found: contracts, company, products, employees


In [22]:
for chunk in chunks:
    if 'CEO' in chunk.page_content:
        print(chunk)
        print("_________")

page_content='# Avery Lancaster

## Summary
- **Date of Birth**: March 15, 1985  
- **Job Title**: Co-Founder & Chief Executive Officer (CEO)  
- **Location**: San Francisco, California  

## Insurellm Career Progression
- **2015 - Present**: Co-Founder & CEO  
  Avery Lancaster co-founded Insurellm in 2015 and has since guided the company to its current position as a leading Insurance Tech provider. Avery is known for her innovative leadership strategies and risk management expertise that have catapulted the company into the mainstream insurance market.  

- **2013 - 2015**: Senior Product Manager at Innovate Insurance Solutions  
  Before launching Insurellm, Avery was a leading Senior Product Manager at Innovate Insurance Solutions, where she developed groundbreaking insurance products aimed at the tech sector.' metadata={'source': '/app/jupyter_notebook_ai_clone/llm_udemy/llm_engineering/week5/knowledge-base/employees/Avery Lancaster.md', 'doc_type': 'employees'}
_________
page_con